# LangChain Integration: 3-Line Setup

## The Integration Pattern

Add Relay governance to LangChain agents with minimal code changes:

```python
# Before Relay
result = stripe.Charge.create(amount=5000)

# After Relay (just add policy check)
relay.validate(manifest)  # 3 lines of code
result = stripe.Charge.create(amount=5000)
```

**No refactoring required** - just middleware between decision and execution.

## Setup

In [ ]:
import sys
sys.path.insert(0, '../')

import requests
import json
from datetime import datetime, UTC

# Verify Relay is running
try:
    requests.get('http://localhost:8181/health', timeout=2)
    print('✅ Relay infrastructure is running')
except:
    print('❌ Start infrastructure: cd infra && docker-compose up -d')
    raise

## Step 1: Traditional Tool (No Protection)

A standard LangChain tool without governance.

In [ ]:
def create_payment_unprotected(amount, customer):
    """Unprotected payment tool."""
    print(f"💳 Payment executed: ${amount/100:.2f}")
    return {"id": "ch_123", "amount": amount, "status": "succeeded"}

# Test with unauthorized amount
print("❌ UNPROTECTED TOOL:")
print("─" * 70)
result = create_payment_unprotected(7500, "customer_123")  # $75 exceeds limit
print("⚠️  $75 payment succeeded (no policy check!)")

## Step 2: Add Relay Protection

Wrap the tool with policy validation - only 3 lines added.

In [ ]:
class PolicyViolationError(Exception):
    """Raised when policy denies an action."""
    pass

def create_payment_protected(amount, customer, agent_id="langchain-agent"):
    """Protected payment tool with Relay validation."""
    
    # BUILD MANIFEST (1 line in production with SDK)
    manifest = {
        "agent": {"agent_id": agent_id, "org_id": "demo"},
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": amount}
        }
    }
    
    # VALIDATE WITH RELAY (1 line)
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': manifest}
    )
    
    # CHECK RESULT (1 line)
    result = response.json()['result']
    if not result.get('allow', False):
        raise PolicyViolationError(result.get('reason', 'Policy denied'))
    
    # EXECUTE (original code)
    print(f"💳 Payment executed: ${amount/100:.2f}")
    return {"id": "ch_456", "amount": amount, "status": "succeeded"}

# Test with same unauthorized amount
print("\n✅ PROTECTED TOOL:")
print("─" * 70)
try:
    result = create_payment_protected(7500, "customer_123")  # $75 exceeds limit
    print("Payment succeeded")
except PolicyViolationError as e:
    print(f"🚫 Blocked: {e}")

# Test with authorized amount
try:
    result = create_payment_protected(3500, "customer_456")  # $35 within limit
    print("✅ Approved and executed")
except PolicyViolationError as e:
    print(f"Blocked: {e}")

## Step 3: Integration with LangChain

Using Relay-protected tools in a LangChain agent.

In [ ]:
# Mock LangChain structures for demo
class Tool:
    def __init__(self, name, func, description):
        self.name = name
        self.func = func
        self.description = description
    
    def run(self, **kwargs):
        return self.func(**kwargs)

# Create protected tool
payment_tool = Tool(
    name="create_payment",
    func=lambda **kwargs: create_payment_protected(**kwargs),
    description="Create payment (Relay-protected, $50 limit)"
)

# Simulate agent using tool
test_cases = [
    {"desc": "Small payment", "amount": 2500, "customer": "cus_001"},
    {"desc": "Large payment", "amount": 7500, "customer": "cus_002"},
    {"desc": "Medium payment", "amount": 4800, "customer": "cus_003"},
]

print("\nAgent Tool Usage:")
print("=" * 70)

for test in test_cases:
    print(f"\n{test['desc']} (${test['amount']/100:.2f}):")
    try:
        payment_tool.run(amount=test['amount'], customer=test['customer'])
        print("  ✅ Executed")
    except PolicyViolationError as e:
        print(f"  🚫 {e}")

## Production Integration Patterns

### Pattern 1: Middleware Wrapper

```python
def relay_protect(provider, method):
    """Decorator to add Relay protection to any function."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            # Build manifest
            manifest = build_manifest(provider, method, *args, **kwargs)
            
            # Validate
            if not relay.validate(manifest):
                raise PolicyViolationError("Denied by policy")
            
            # Execute
            return func(*args, **kwargs)
        return wrapper
    return decorator

@relay_protect(provider="stripe", method="create_payment")
def create_payment(amount, customer):
    return stripe.Charge.create(amount=amount, customer=customer)
```

### Pattern 2: Tool Wrapper

```python
def wrap_with_relay(langchain_tool, provider, method):
    """Wrap existing LangChain tool with Relay."""
    original_func = langchain_tool.func
    
    def protected_func(**kwargs):
        manifest = build_manifest(provider, method, **kwargs)
        if not relay.validate(manifest):
            raise PolicyViolationError("Denied")
        return original_func(**kwargs)
    
    langchain_tool.func = protected_func
    return langchain_tool

# Apply to existing tool
protected_tool = wrap_with_relay(
    my_payment_tool,
    provider="stripe",
    method="create_payment"
)
```

### Pattern 3: Agent-Level Protection

```python
class RelayProtectedAgent:
    """Wrap entire LangChain agent."""
    
    def __init__(self, agent, relay_client):
        self.agent = agent
        self.relay = relay_client
        
        # Protect all tools
        for tool in agent.tools:
            tool.func = self._protect(tool.func, tool.name)
    
    def _protect(self, func, tool_name):
        def wrapper(**kwargs):
            # Auto-validate based on tool metadata
            if not self.relay.validate_tool(tool_name, kwargs):
                raise PolicyViolationError("Denied")
            return func(**kwargs)
        return wrapper
```

## Key Benefits

### 1. Minimal Code Changes
- Add 3 lines per tool (build, validate, check)
- No refactoring of existing logic
- Works with existing LangChain code

### 2. Framework Agnostic
- Same pattern for LangChain, CrewAI, AutoGPT
- Middleware approach = universal
- Not tied to specific frameworks

### 3. Transparent to LLM
- LLM doesn't need to know about policies
- No prompt engineering required
- Works with any model (GPT-4, Claude, Llama)

### 4. Fail-Safe
- If Relay is down, actions blocked (fail-closed)
- Cannot be bypassed by prompting
- Cryptographic audit trail

## Comparison

| Aspect | Without Relay | With Relay |
|--------|--------------|------------|
| Code changes | - | +3 lines per tool |
| Policy enforcement | ❌ None | ✅ Automatic |
| Audit trail | ❌ None | ✅ Complete |
| Manipulation protection | ❌ Vulnerable | ✅ Protected |
| Compliance ready | ❌ No | ✅ Yes |

## Next Steps

1. **[04_company_policies.ipynb](04_company_policies.ipynb)** - Learn policy authoring
2. **[05_real_world_scenarios.ipynb](05_real_world_scenarios.ipynb)** - See production use cases
3. **[02_adversarial_prompt_protection.ipynb](02_adversarial_prompt_protection.ipynb)** - Understand security